In [83]:
import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# import warnings
# warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)

## Подготовка данных

In [64]:
df_overpass_1 = pd.read_csv('../../checkpoint-2/overpass/adj_df_additional.csv',index_col=[0])
df_overpass_2 = pd.read_csv('../../checkpoint-2/overpass/adj_df.csv',index_col=[0])
df_overpass = df_overpass_1.merge(df_overpass_2, on = 'id')
df_info = pd.read_csv('../../checkpoint-2/overpass/concat_df.csv',index_col=[0])
df = df_info.merge(df_overpass, on = ['id'])

meaningful_subset = set(df.columns).difference(['id', 'long', 'lat', 'utm',
                                                'geometry', 'address', 'address_rus'])

train = df[~df.target.isna()][meaningful_subset]
test = df[df.target.isna()][meaningful_subset].drop(columns = ['target'])

/var/folders/8m/fpt3kb2n0ys89v3nl34fszrd2jtv8f/T/ipykernel_30030/819544030.py:10: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  train = df[~df.target.isna()][meaningful_subset]
/var/folders/8m/fpt3kb2n0ys89v3nl34fszrd2jtv8f/T/ipykernel_30030/819544030.py:11: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  test = df[df.target.isna()][meaningful_subset].drop(columns = ['target'])


## ОНЕ и Нормировка

In [65]:
enc = OneHotEncoder(handle_unknown='ignore',drop = 'first')
enc.fit(train.atm_group.to_numpy().reshape(-1, 1))
train[enc.get_feature_names_out(['atm_group'])] = pd.DataFrame(enc.transform(train.atm_group.to_numpy().reshape(-1, 1)).toarray())
test[enc.get_feature_names_out(['atm_group'])] = pd.DataFrame(enc.transform(test.atm_group.to_numpy().reshape(-1, 1)).toarray())

train = train.drop(columns=['atm_group'])
test = test.drop(columns=['atm_group'])


In [66]:
X_train, X_test, y_train, y_test = train_test_split(train.drop(columns = ['target']),train.target , test_size=0.8, random_state=42)


In [67]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = pd.DataFrame(scaler.transform(X_train),columns = X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test),columns = X_test.columns)
test = pd.DataFrame(scaler.transform(test),columns = test.columns)


## Baseline средним/медианным значением

In [98]:
print('rmse median baseline:',mean_squared_error(y_test, [y_test.median()]*y_test.size, squared=False))
print('rmse mean baseline:',mean_squared_error(y_test, [y_test.mean()]*y_test.size, squared=False))

rmse median baseline: 0.08768333387561215
rmse mean baseline: 0.08614842782465978


## Применение ridge регрессии

In [93]:
ridge = Ridge(alpha = 0.05).fit(X_train, y_train)
y_pred = ridge.predict(X_test)
print('rmse baseline:',mean_squared_error(y_test, y_pred, squared=False))

rmse baseline: 0.04948708519299563


## Наш baseline rmse: 0.049